# Dataset
```
a b c | a
x y z | x
1 2 3 | 1
```

In [1322]:
import string
pool = string.ascii_letters + string.digits
pool_size = len(pool)
pool, pool_size

('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789', 62)

In [1323]:
stoi = {s:i for i,s in enumerate(pool)}
itos = {i:s for s,i in stoi.items()}
encode = lambda str: [stoi[c] for c in str]
decode = lambda ints: ''.join([itos[i] for i in ints])
x = encode('john')
x, decode(x)


([9, 14, 7, 13], 'john')

In [1324]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [1325]:
row_count = 1024
col_count = 3
dataset = torch.randint(0, len(pool), (row_count,col_count))
X=dataset
#Y=dataset[:, 0].unsqueeze(dim=1)
Y=dataset[:, 0]

x=0
for row in dataset[:5]:
  print(f"{decode(row.tolist())} -> {itos[Y[x].tolist()]}")
  x+=1

X.shape, X.dtype, Y.shape, Y.dtype

HQV -> H
v2D -> v
ke0 -> k
kUh -> k
rKW -> r


(torch.Size([1024, 3]), torch.int64, torch.Size([1024]), torch.int64)

In [1326]:
X.shape, X[19]

(torch.Size([1024, 3]), tensor([34, 10, 27]))

In [1327]:
n_embd = 10
class SomeModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embeddings = nn.Embedding(pool_size, n_embd)
    self.pos_embeddings = nn.Embedding(3, n_embd)
    self.l1 = nn.Linear(n_embd, pool_size)

  def forward(self, idx, targets=None):
    B,T = idx.shape
    tok_emb = self.token_embeddings(idx) # (B,T,C)
    pos_emb = self.pos_embeddings(torch.arange(T))
    x = tok_emb + pos_emb
    x = self.l1(x) # (B,T, pool_size)
    x = x.mean(dim=1) # (B, pool_size)
    if targets is None:
      return x, None
    loss = F.cross_entropy(x, targets)
    return x, loss
  
  def generate(self, idx):
    logits, _ = self(idx)
    probs = F.softmax(logits, dim=1)
    hit = torch.multinomial(probs, num_samples=1)
    return hit

    


In [1328]:
m = SomeModel()
optimizer = torch.optim.AdamW(params=m.parameters(), lr=0.1)


In [ ]:
for _ in range(1000):
  logits, loss=m(X, Y)

  optimizer.zero_grad(set_to_none=None)
  loss.backward()
  optimizer.step()

print(loss)

given=torch.tensor(encode('abc')).unsqueeze(dim=0)
out=[]
for _ in range(20):
  hit = m.generate(given)[0].item()
  out.append(itos[hit])
print(f'answers (we expect a): {"".join(out)}')


given=torch.tensor(encode('834')).unsqueeze(dim=0)
out=[]
for _ in range(20):
  hit = m.generate(given)[0].item()
  out.append(itos[hit])
print(f'answers (we expect 8): {"".join(out)}')

# below result is not taken one-shot

tensor(0.0431, grad_fn=<NllLossBackward0>)
answers (we expect a): aaaaaaaaaaaaaaaaaaaa
answers (we expect 8): 88888888888888888888


### Notes
- v001 - its not learning sequence/order because we don't have it in our model, idk, peaky, and noisy
- v002 - `x = x.mean(dim=1)` averaging seems to point to a more smooth distribution of probabilities? more consistent single answers? meaning it knows that a single char is the right answer
- v003 - adding + pos embedding, kinda funny as it seems to have done nothing